## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-28-18-47-56 +0000,nyt,Syrian Protests Over Lack of Security Leave 2 ...,https://www.nytimes.com/2025/12/28/world/middl...
1,2025-12-28-18-47-25 +0000,nyt,Gmail to Let Users Change Their Addresses Whil...,https://www.nytimes.com/2025/12/28/technology/...
2,2025-12-28-18-44-00 +0000,wsj,Zelensky Meets With Trump on Peace Deal Amid L...,https://www.wsj.com/world/trump-zelensky-russi...
3,2025-12-28-18-42-41 +0000,nypost,IDF gets first combat-ready laser interception...,https://nypost.com/2025/12/28/world-news/idf-g...
4,2025-12-28-18-27-58 +0000,nypost,Thieves made off with $400K worth of Costco lo...,https://nypost.com/2025/12/28/us-news/thieves-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2367/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
16,trump,19
39,new,13
63,brigitte,8
64,bardot,8
14,zelensky,7


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
98,2025-12-28-05-53-06 +0000,bbc,Zelensky says Russia 'doesn't want peace' ahea...,https://www.bbc.com/news/articles/c1dzvpg1341o...,51
14,2025-12-28-17-32-52 +0000,nyt,Zelensky to Meet With Trump at Mar-a-Lago Abou...,https://www.nytimes.com/2025/12/28/world/europ...,48
31,2025-12-28-15-51-54 +0000,nypost,Trump touts ‘good and very productive’ call wi...,https://nypost.com/2025/12/28/us-news/zelensky...,47
41,2025-12-28-15-13-35 +0000,nypost,Trump declares cease-fire between Thailand and...,https://nypost.com/2025/12/28/us-news/trump-de...,45
2,2025-12-28-18-44-00 +0000,wsj,Zelensky Meets With Trump on Peace Deal Amid L...,https://www.wsj.com/world/trump-zelensky-russi...,44


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
98,51,2025-12-28-05-53-06 +0000,bbc,Zelensky says Russia 'doesn't want peace' ahea...,https://www.bbc.com/news/articles/c1dzvpg1341o...
53,42,2025-12-28-13-00-00 +0000,startribune,"New year brings new laws, including paid famil...",https://www.startribune.com/new-laws-minnesota...
46,38,2025-12-28-14-17-31 +0000,cbc,"Brigitte Bardot, icon of French cinema turned ...",https://www.cbc.ca/news/entertainment/brigitte...
93,32,2025-12-28-08-59-41 +0000,nypost,Next major storm system threatens Midwest with...,https://nypost.com/2025/12/28/us-news/major-st...
124,30,2025-12-28-00-11-17 +0000,cbc,"Amid a civil war, polls open in Myanmar for 1s...",https://www.cbc.ca/news/world/myanmar-election...
103,26,2025-12-28-04-51-49 +0000,nypost,Gov. Tim Walz ‘needs to be held accountable’ a...,https://nypost.com/2025/12/27/us-news/minnesot...
100,24,2025-12-28-05-33-21 +0000,nypost,"Australian cruise ship runs aground, strikes c...",https://nypost.com/2025/12/28/world-news/austr...
127,23,2025-12-28-00-00-00 +0000,wsj,President Trump overrode some on the religious...,https://www.wsj.com/politics/policy/how-trump-...
94,23,2025-12-28-08-50-52 +0000,nypost,Utah mom Elleshia Seymour allegedly abducted h...,https://nypost.com/2025/12/28/us-news/utah-mom...
24,20,2025-12-28-16-43-39 +0000,nypost,Oklahoma man hit with manslaughter charge afte...,https://nypost.com/2025/12/28/us-news/oklahoma...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
